In [1]:
import torch

In [2]:
import scipy.io
import numpy as np

## DE Features (one subject)

https://github.com/ynulonger/DE_CNN

https://www.researchgate.net/publication/328504085_Continuous_Convolutional_Neural_Network_with_3D_Input_for_EEG-Based_Emotion_Recognition

In [3]:
deap_de_path = '../../methods/DE_CNN/1D_dataset/'

In [4]:
s0 = scipy.io.loadmat(deap_de_path + 'DE_s01.mat')
for i, key in enumerate(s0):
    print(key)

__header__
__version__
__globals__
base_data
data
valence_labels
arousal_labels


In [5]:
X_0 = s0['data']
y_0_valence = s0['valence_labels']
y_0_arousal = s0['arousal_labels']

In [6]:
X_0.shape

(2400, 4, 32)

In [7]:
y_0_valence.shape

(1, 2400)

In [8]:
np.transpose(y_0_valence).shape

(2400, 1)

### Merge all subjects' features

Subjects number and indexing

In [9]:
c = 2400

for idx in range(32):
    print(idx+1, idx*c, (idx+1)*c-1)

1 0 2399
2 2400 4799
3 4800 7199
4 7200 9599
5 9600 11999
6 12000 14399
7 14400 16799
8 16800 19199
9 19200 21599
10 21600 23999
11 24000 26399
12 26400 28799
13 28800 31199
14 31200 33599
15 33600 35999
16 36000 38399
17 38400 40799
18 40800 43199
19 43200 45599
20 45600 47999
21 48000 50399
22 50400 52799
23 52800 55199
24 55200 57599
25 57600 59999
26 60000 62399
27 62400 64799
28 64800 67199
29 67200 69599
30 69600 71999
31 72000 74399
32 74400 76799


In [10]:
deap_de_path

'../../methods/DE_CNN/1D_dataset/'

In [11]:
merge_de_cnn_features = False

In [12]:
if merge_de_cnn_features:
    de_cnn_features = np.empty((2400 * 32, 4, 32))
    de_cnn_y_valence = np.empty((2400 * 32, 1))
    de_cnn_y_arousal = np.empty((2400 * 32, 1))
    
    for i in range(1, 33):  # Subjects 1-32 in DEAP
        subj_data = scipy.io.loadmat(deap_de_path + f'DE_s{i:02}.mat')

        Xi_de = subj_data['data']
        yi_valence = np.transpose(subj_data['valence_labels'])
        yi_arousal = np.transpose(subj_data['arousal_labels'])
        
        idx = i-1  # indexing 0-31 for arrays
        c = 2400  # size of each subject's trials*1s_windows

        # efficient assigning, not really needed, could use np.append
        de_cnn_features[idx*c:(idx+1)*c] = Xi_de
        de_cnn_y_valence[idx*c:(idx+1)*c] = yi_valence
        de_cnn_y_arousal[idx*c:(idx+1)*c] = yi_arousal
        
        save_dict = {'data': de_cnn_features, 
                     'valence_labels': de_cnn_y_valence, 
                     'arousal_labels': de_cnn_y_arousal}
         
    np.save(deap_de_path + 'DE_merged.npy', save_dict)  

In [13]:
if not merge_de_cnn_features:
    de_cnn_merged = np.load(deap_de_path + 'DE_merged.npy', allow_pickle=True).item()
    de_cnn_features = de_cnn_merged['data']
    de_cnn_y_valence = de_cnn_merged['valence_labels']
    de_cnn_y_arousal = de_cnn_merged['arousal_labels']
    
    print('Loaded from file.')
    print(de_cnn_features.shape)
    print(de_cnn_y_valence.shape)
    print(de_cnn_y_arousal.shape)

Loaded from file.
(76800, 4, 32)
(76800, 1)
(76800, 1)


## Load DE Features (all subjects)

https://github.com/gzoumpourlis/DEAP_MNE_preprocessing

In [14]:
de_features_path = '../../preprocessing/DEAP_MNE_preprocessing/features_new/de_feats_merged.npy'

In [15]:
de_features = np.load(de_features_path)

In [16]:
de_features.shape

(1280, 32, 5, 232)

In [17]:
deap_path = '../../datasets/DEAP/merged/'

In [18]:
y = np.load(deap_path + 'deap_full_labels.npy')
y.shape

(1280, 3)

Column 0 is Valence, 1 is Arousal, 2 is quadrants notation (HAHV, HALV, LAHV, LALV)

In [19]:
valence = 0
arousal = 1
quadrants = 2

In [20]:
y = y[:, valence]

In [21]:
y.shape

(1280,)

## Define DEAP Dataset

In [22]:
from torch.utils.data import Dataset

In [23]:
class DEAPDataset(Dataset):
    def __init__(self, data, labels):
        self.X = data
        self.y = labels
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

Either use 'de_features' (DEAP_MNE_preprocessing) or 'de_cnn_features' (DE_CNN)

Labels are 'y' or 'de_cnn_y_valence/de_cnn_y_arousal' respectively

In [24]:
use_de_cnn = True

In [25]:
if use_de_cnn:
    deap_dataset = DEAPDataset(de_cnn_features, de_cnn_y_valence)
else:
    deap_dataset = DEAPDataset(de_features, y)

In [26]:
deap_dataset[0][0].shape

(4, 32)

In [27]:
input_data = de_features[:64] # small batch deap_mne_preprocessing
input_data.shape

(64, 32, 5, 232)

In [28]:
input_data = de_cnn_features[:64] # small batch de_cnn
input_data.shape

(64, 4, 32)

## DataLoader

In [29]:
from torch.utils.data import DataLoader

In [30]:
batch_size=32

In [31]:
train_split = 0.75
test_split = 0.25

train_n_elems = int(train_split * len(deap_dataset))
test_n_elems = int(test_split * len(deap_dataset))

print(train_split, test_split)
print(train_n_elems, test_n_elems)

0.75 0.25
57600 19200


In [53]:
from torch.utils.data import random_split

In [58]:
deap_train, deap_test = random_split(deap_dataset, [train_n_elems, test_n_elems])

In [59]:
# deap_train = deap_dataset[:train_n_elems]
# deap_test = deap_dataset[train_n_elems:]

In [60]:
len(deap_train)

57600

In [61]:
train_dataloader = DataLoader(deap_train, batch_size=batch_size, shuffle=False, num_workers=0)
test_dataloader = DataLoader(deap_test, batch_size=batch_size, shuffle=False, num_workers=0)

## Define BiHDM Model

In [62]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
device

device(type='cpu')

In [63]:
from Models_DEAP import BiHDM

### BiHDM Initialization Parameters

In [64]:
hidden_size = 32
num_layers = 2
input_size = 4
n_classes = 1

# batch_first=False
# bidirectional=False

fc_input=448
fc_hidden=96

In [65]:
model = BiHDM(hidden_size=hidden_size, num_layers=num_layers, input_size=input_size, 
              fc_input=fc_input, fc_hidden=fc_hidden, n_classes=n_classes)

In [66]:
model.to(device).float()

BiHDM(
  (RNN_VL): RNN(4, 32, num_layers=2)
  (RNN_VR): RNN(4, 32, num_layers=2)
  (RNN_V): RNN(32, 32, num_layers=2)
  (RNN_HL): RNN(4, 32, num_layers=2)
  (RNN_HR): RNN(4, 32, num_layers=2)
  (RNN_H): RNN(32, 32, num_layers=2)
  (fc_v): Sequential(
    (0): Linear(in_features=448, out_features=96, bias=True)
    (1): ReLU()
  )
  (fc_h): Sequential(
    (0): Linear(in_features=448, out_features=96, bias=True)
    (1): ReLU()
  )
  (fc_c): Sequential(
    (0): Linear(in_features=96, out_features=1, bias=True)
  )
)

## Training BiHDM

In [67]:
criterion = torch.nn.BCEWithLogitsLoss()

In [68]:
lr=0.001
betas=(0.9, 0.999)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)

In [108]:
num_epochs = 200

In [109]:
n_total_steps = len(train_dataloader)

In [110]:
model_path = '../BiHDM-Model.pth'

In [111]:
skip_training = False
resume_training = True

In [113]:
if not skip_training:
    if resume_training:
        #checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict']) 
        min_loss = checkpoint['min_loss']
    else:
        min_loss = 1000
        
print(min_loss)    

0.10696408897638321


In [114]:
if not skip_training:
    for epoch in range(num_epochs):
        for i, (data, labels) in enumerate(train_dataloader):
            model.train()

            data = data.to(device).float().permute(0, 2, 1)
            labels = labels.to(device).float()

            outputs = model(data)    
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 

            if (i+1) % 600 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.6f}')

            if epoch > 5 and loss.item() < min_loss:
                    min_loss = loss.item()
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'min_loss': min_loss
                    }, model_path)
                    print(f'Saved checkpoint - loss: {min_loss:.6f}')

    print('Finished Training')

Epoch [1/200], Step [600/1800], Loss: 0.516962
Epoch [1/200], Step [1200/1800], Loss: 0.573717
Epoch [1/200], Step [1800/1800], Loss: 0.341625
Epoch [2/200], Step [600/1800], Loss: 0.608397
Epoch [2/200], Step [1200/1800], Loss: 0.646369
Epoch [2/200], Step [1800/1800], Loss: 0.301434
Epoch [3/200], Step [600/1800], Loss: 0.544228
Epoch [3/200], Step [1200/1800], Loss: 0.680549
Epoch [3/200], Step [1800/1800], Loss: 0.309841
Epoch [4/200], Step [600/1800], Loss: 0.517641
Epoch [4/200], Step [1200/1800], Loss: 0.733751
Epoch [4/200], Step [1800/1800], Loss: 0.272661
Epoch [5/200], Step [600/1800], Loss: 0.507598
Epoch [5/200], Step [1200/1800], Loss: 0.658273
Epoch [5/200], Step [1800/1800], Loss: 0.341298
Epoch [6/200], Step [600/1800], Loss: 0.476387
Epoch [6/200], Step [1200/1800], Loss: 0.651253
Epoch [6/200], Step [1800/1800], Loss: 0.279729
Epoch [7/200], Step [600/1800], Loss: 0.667901
Epoch [7/200], Step [1200/1800], Loss: 0.596846
Epoch [7/200], Step [1800/1800], Loss: 0.294546

Epoch [55/200], Step [600/1800], Loss: 0.405220
Epoch [55/200], Step [1200/1800], Loss: 0.525293
Epoch [55/200], Step [1800/1800], Loss: 0.170830
Epoch [56/200], Step [600/1800], Loss: 0.564863
Epoch [56/200], Step [1200/1800], Loss: 0.244268
Epoch [56/200], Step [1800/1800], Loss: 0.262767
Epoch [57/200], Step [600/1800], Loss: 0.338160
Epoch [57/200], Step [1200/1800], Loss: 0.368122
Epoch [57/200], Step [1800/1800], Loss: 0.201050
Epoch [58/200], Step [600/1800], Loss: 0.571804
Epoch [58/200], Step [1200/1800], Loss: 0.297761
Epoch [58/200], Step [1800/1800], Loss: 0.081477
Epoch [59/200], Step [600/1800], Loss: 0.414562
Epoch [59/200], Step [1200/1800], Loss: 0.352077
Epoch [59/200], Step [1800/1800], Loss: 0.140315
Epoch [60/200], Step [600/1800], Loss: 0.389959
Epoch [60/200], Step [1200/1800], Loss: 0.427009
Epoch [60/200], Step [1800/1800], Loss: 0.174780
Epoch [61/200], Step [600/1800], Loss: 0.431297
Epoch [61/200], Step [1200/1800], Loss: 0.328937
Epoch [61/200], Step [1800/

KeyboardInterrupt: 

## Test Accuracy

In [115]:
load_model = True

In [116]:
chosen_n_classes = 1

In [117]:
with torch.no_grad():
    model.eval()  
    if load_model:
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        
    n_correct = 0
    n_samples = 0

    for data, labels in test_dataloader:
        data = data.to(device).float().permute(0, 2, 1)
        labels = labels.to(device).float().reshape(-1,)

        outputs = model(data)

        # torch.max returns (value, index)
        _, predicted = torch.max(outputs, 1)

        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

acc = n_correct / n_samples * 100.0 
print(f'Accuracy of the network: {acc:.2f}%')

Accuracy of the network: 44.48%
